In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, StratifiedShuffleSplit,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, average_precision_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.utils import Bunch
from scipy.stats import randint, loguniform

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
import warnings
from sklearn.model_selection import KFold, cross_val_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('silver_stock_info.csv')

In [3]:
modelDF = df[['open', 'high', 'low', 'close', 'volume','movement']]


CONFIG = {
    "test_size": 0.30,
    "random_state": 18,
    "cv_folds": 10,
}

In [4]:
modelDF['high_range'] = modelDF['high'] - modelDF['low']
modelDF['end_price'] = modelDF['close'] - modelDF['open']

C:\Users\yluko\AppData\Local\Temp\ipykernel_29536\646972467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelDF['high_range'] = modelDF['high'] - modelDF['low']
C:\Users\yluko\AppData\Local\Temp\ipykernel_29536\646972467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelDF['end_price'] = modelDF['close'] - modelDF['open']


In [6]:
numeric_cols = modelDF.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in modelDF.columns if c not in numeric_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore",sparse_output=False), categorical_cols)
    ],
    remainder="drop"
)
preprocess.set_output(transform="pandas")

transformed_X = preprocess.fit_transform(modelDF)

target = transformed_X['num__movement'].astype(int)
features = transformed_X[['num__volume','num__high_range','num__end_price']]


X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=.30, random_state=18, stratify=target)




cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=18)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Numeric cols: {len(numeric_cols)}, Categorical cols: {len(categorical_cols)}")
transformed_X
vif_data = pd.DataFrame()
features

Train shape: (175, 3), Test shape: (75, 3)
Numeric cols: 8, Categorical cols: 0


,num__volume,num__high_range,num__end_price
0,0.201621,1.109206,0.853620
1,0.071125,-0.324436,0.409500
2,1.713295,2.654665,-3.381865
3,-0.191770,-0.352390,-0.430676
4,-0.406593,-0.526104,0.469100
...,...,...,...
245,-0.826218,-0.434255,-0.057692
246,-0.395570,-0.705808,0.067277
247,-0.851786,-0.492160,-0.659465
248,-1.022024,-0.511129,0.419113


In [ ]:

vif_data = pd.DataFrame()
vif_data["feature"] = features.columns
vif_data["VIF"] = [variance_inflation_factor(features.values, i) for i in range(features.shape[1])]
print(vif_data)


           feature       VIF
0      num__volume  1.579244
1  num__high_range  1.598859
2   num__end_price  1.016243


In [7]:
tree_clf = DecisionTreeClassifier(
    random_state=42,
    max_depth=5,
    class_weight="balanced"
)

tree_clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight='balanced', max_depth=5, random_state=42)

In [10]:
report = classification_report(y_test, tree_clf.predict(X_test), zero_division=0)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        47
           1       0.97      1.00      0.98        28

    accuracy                           0.99        75
   macro avg       0.98      0.99      0.99        75
weighted avg       0.99      0.99      0.99        75

